### Auto check with authentication (set file to be privately accessible).

In [2]:
import os
from dotenv import load_dotenv
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
import datetime
import pickle

# Load environment variables
load_dotenv()

SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
#LOCAL_FILE_PATH = r"D:\GithubDEXON\2024_026_local\file_version.txt"
GOOGLE_DRIVE_FILE_ID = "1tuEWr7LQ5_6CRzLxjghG4tNvvekKa0ZA"

# Function to get the path from the file name.
def find_file(search_dir, file_name):
    for root, dirs, files in os.walk(search_dir):
        if file_name in files:
            return os.path.join(root, file_name)
    return None

# Function to get credentials for the Google Drive API.
def get_credentials():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_config(
                {"installed":{"client_id": os.getenv('CLIENT_ID'),
                              "project_id":os.getenv('PROJECT_ID'),
                              "token_uri":"https://oauth2.googleapis.com/token",
                              "auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs",
                              "client_secret":os.getenv('CLIENT_SECRET'),
                              "redirect_uris":["http://localhost"]
                             }
                },
                SCOPES
            )
            creds = flow.run_local_server(port=0)
        
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    
    return creds

# Function to compare a local file with a file on Google Drive.
def check_file_update(file_path):
    try:
        creds = get_credentials()
        service = build('drive', 'v3', credentials=creds)
        
        # Get local file modification time
        local_mod_time = datetime.datetime.fromtimestamp(os.path.getmtime(file_path))
        local_mod_time = local_mod_time.replace(microsecond=0)  # Remove microseconds for comparison
        
       # Get Google Drive file
        drive_file = service.files().get(fileId=GOOGLE_DRIVE_FILE_ID, fields="modifiedTime").execute()
        drive_mod_time = datetime.datetime.strptime(drive_file['modifiedTime'], '%Y-%m-%dT%H:%M:%S.%fZ')
        drive_mod_time = drive_mod_time.replace(tzinfo=None, microsecond=0)  # Remove timezone and microseconds

        print(f"Local file last modified: {local_mod_time}")
        print(f"Google Drive file last modified: {drive_mod_time}")

        if local_mod_time > drive_mod_time:
            print("Local file is newer")
        elif local_mod_time < drive_mod_time:
            print("Google Drive file is newer")
        else:
            print("Both files have the same modification time")
    except Exception as e:
        print(f"An error occurred: {e}")

def main():

    file_name = "file_version.txt"
    search_dir = "D:\\autoUpdateCheck"
    
    LOCAL_FILE_PATH = find_file(search_dir, file_name)
    print("file name: ", LOCAL_FILE_PATH)
    if LOCAL_FILE_PATH:
        print("file name: ", LOCAL_FILE_PATH)
        check_file_update(LOCAL_FILE_PATH)
    else:
        print("File not found.")
        
    # Keep the cmd window open
    os.system("pause")

if __name__ == "__main__":
    main()


file name:  None
File not found.


### Auto Check without authentication (set file to be publicly accessible)

In [4]:
import os
import datetime
import requests
import pickle

# Google Drive public link
GOOGLE_DRIVE_LINK = "https://drive.google.com/uc?export=download&id=1tuEWr7LQ5_6CRzLxjghG4tNvvekKa0ZA"

# Function to get the path from the file name.
def find_file(search_dir, file_name):
    for root, dirs, files in os.walk(search_dir):
        if file_name in files:
            return os.path.join(root, file_name)
    return None

# Function to compare a local file with a file on Google Drive.
def check_file_update(file_path):
    try:
        # Get local file modification time
        local_mod_time = datetime.datetime.fromtimestamp(os.path.getmtime(file_path))
        local_mod_time = local_mod_time.replace(microsecond=0)  # Remove microseconds for comparison
        
        # Get Google Drive file
        response = requests.head(GOOGLE_DRIVE_LINK, allow_redirects=True)
        if response.status_code == 200:
            drive_mod_time = datetime.datetime.strptime(response.headers['Last-Modified'], '%a, %d %b %Y %H:%M:%S %Z')
            drive_mod_time = drive_mod_time.replace(tzinfo=None)

            print(f"Local file last modified: {local_mod_time}")
            print(f"Google Drive file last modified: {drive_mod_time}")

            if local_mod_time > drive_mod_time:
                print("Local file is newer")
            elif local_mod_time < drive_mod_time:
                print("Google Drive file is newer")
            else:
                print("Both files have the same modification time")
        else:
            print(f"Failed to access Google Drive file. Status code: {response.status_code}")

    except Exception as e:
        print(f"An error occurred: {e}")

def main():
    # Local file path
    file_name = "file_version.txt"
    search_dir = "D:\\autoUpdateCheck"
    LOCAL_FILE_PATH = find_file(search_dir, file_name)
    
    if LOCAL_FILE_PATH:
        print("file name: ", LOCAL_FILE_PATH)
        check_file_update(LOCAL_FILE_PATH)
    else:
        print("File not found.")
    
    # Keep the cmd window open
    os.system("pause")

if __name__ == "__main__":
    main()


file name:  D:\autoUpdateCheck\file_version.txt
Local file last modified: 2024-07-19 10:17:40
Google Drive file last modified: 2024-07-18 08:03:22
Local file is newer


### Python script to an executable (EXE) file

run cmd: pyinstaller --onefile --icon=dexon-icon.jpg autoUpdate.py